In [145]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import geopandas as gpd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

import yaml

from paths import *
from scripts import manipulation
from scripts import io

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, offline
from scripts import vis_layout

from os import listdir
import glob

import datetime
today = datetime.datetime.today().strftime('%Y_%m_%d')

import requests

In [159]:
df = pd.read_excel('http://iabas.org.br/wp-content/uploads/2020/06/ctrcovid19.xls')

df.to_excel(f'../data/oss_individuais/iabas/{today}_iabas.xlsx', index=False)

In [160]:
df.columns = manipulation.normalize_cols(df.columns)
df['cnpj'] = df['cnpj'].str.replace('.','').str.replace('-','').str.replace('/','')

df['valor'] = df['valor'].astype(str).apply(lambda x: x.replace('R$ 1.934,942,80','1934942.80') if 'R$ 1.934,942,80' in x else x).astype(float)

In [161]:
mask = df['cnpj']=='29284919000136'

df[mask]

,forma_de_contratacao,data_da_contratacao,empresa_contratada,objeto_do_contrato,periodo_da_contratacao,cnpj,valor
17,EMERGENCIAL,2020-04-03,SAÚDE COMPLETA PRONTO SOCORRO E CLINICA MÉDICA LTDA,Contrato emergencial de prestação de serviço médico para atender as necessidades do Hospital Municipal Campanha – Anhembi SP (HMCamp Anhembi),"O prazo de vigência do contrato será pelo prazo de duração do Hospital de Campanha-Anhembi SP (HMCamp)Anhembi), e se findará após sua desativação.",29284919000136,3673063.52
42,EMERGENCIAL,2020-04-15,SAUDE COMPLETA PRONTO SOCORRO,Contrato emergencial de prestação de serviço médico para atendimento do Hospital Bela Vista,90 (noventa dias),29284919000136,1934942.80
69,EMERGENCIAL,2020-05-27,SAUDE COMPLETA PRONTO SOCORRO,Contrato emergencial de prestação de serviço médico para atendimento do Hospital Brasilancia,180 dias ou enquanto durar a pandemia de Covid 19,29284919000136,2459586.50


Socios

In [117]:
df_final = pd.DataFrame()
ne = df['cnpj'].unique().tolist()

while ne!=[]:
    for cnpj in ne:
        url = f'https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj={cnpj}'
    
        r = requests.get(url)

        if r.status_code != 200:
            print(len(ne),' - ',f'{cnpj} nao encontrado')
        else:
            print(len(ne),' - ',url)
            j = r.json()
            dd = pd.DataFrame.from_dict(j['results'])


            df_final = pd.concat([df_final, dd], axis=0)

            ne.remove(cnpj)
            print()

39  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=00331788000119

38  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=05654916000189

37  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=06137183000178

36  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=29227032000106

35  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=24842754000157

34  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=22416258000106

33  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=59519603000173

32  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=27443614000102

31  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=29284919000136

30  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=53689402000101

29  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?cnpj=06088486000148

28  -  https://brasil.io/api/dataset/socios-brasil/socios/data/?c

In [116]:
df_final.to_excel(f'../data/oss_individuais/iabas/{today}_socios_empresas.xlsx', index=False)